In [4]:
import sqlite3
import os
import pandas as pd

In [5]:
# change this if youy have set up an environment variable
working_dir = os.environ.get("FOOTBALL_ANALYTICS")
db_file = os.path.join(working_dir, "db", "football_database.sqlite")

# or use this and input the path where your version of the DB file is stored
# db_file = "C:/.../filepath.sqlite"

# Get SQLite database tables and columns output as a pandas dataframe

In [6]:
# connect to DB
conn = sqlite3.connect(db_file)

# create a cursor object
cursor = conn.cursor()

# SQL query to profile SQLite3 DB
get_db_profile = """
    WITH tables AS (
        SELECT name 
        FROM sqlite_master 
        WHERE type = 'table'
    )
    
    SELECT 
        t.name AS "table", 
        pti.name AS "column"
        -- pti.type AS "column_type",
        -- pti.pk AS "column_is_primary_key"
    FROM tables t
    INNER JOIN pragma_table_info(t.name) pti
    ORDER BY t.name;
"""

# query
cursor.execute(get_db_profile)

# to dataframe
pd.DataFrame(cursor, columns=["table", "column"])

,table,column
0,Country,id
1,Country,name
2,League,id
3,League,country_id
4,League,name
...,...,...
196,Team_Attributes,defenceTeamWidth
197,Team_Attributes,defenceTeamWidthClass
198,Team_Attributes,defenceDefenderLineClass
199,sqlite_sequence,name


# Useful queries to do database profiling with SQLite

In [7]:
# connect to DB
conn = sqlite3.connect(db_file)

# create a cursor object
cursor = conn.cursor()

# list tables
list_tables = "SELECT name FROM sqlite_master WHERE type='table'"

# list columns (position, name, type, number of non null values, default value, primary key indicator)
list_columns_from_table = "PRAGMA table_info(table_name)"

# preview a few rows of data
preview_table_sample = "SELECT * FROM table_name LIMIT 10"

# preview output as dataframe, with column headers
# with SQLite DB, columns aren't available by default
query_output = cursor.execute(preview_table_sample).fetchall()
column_names = [row[0] for row in cursor.description]
df = pd.DataFrame(data=query_output, columns=column_names)

OperationalError: no such table: table_name

# Example of a SQL query with some good practices to consider

- commenting the query
- indenting the code
- avoiding SELECT * statements
- using aliases to name tables

In [ ]:
# retrieving Real Madrid matches
get_rma_matches = """

    -- team_api_id = 8633 is Real Madrid identifier
    -- home and away matches involving Real Madrid will be queried

    SELECT
        m.id, 
        m.country_id, 
        m.league_id,
        m.season,
        m.stage,
        m.date,
        m.match_api_id,
        m.home_team_api_id,
        ht.team_long_name AS home_team_name,
        ht.team_long_name AS home_team_acronym,
        m.away_team_api_id,
        at.team_long_name AS away_team_name,
        at.team_long_name AS away_team_acronym,
        m.home_team_goal,
        m.away_team_goal,
        m.goal,
        m.shoton,
        m.shotoff,
        m.foulcommit,
        m.card,
        m.cross,
        m.corner,
        m.possession
    FROM Match m
    LEFT JOIN Team AS ht
    ON ht.team_api_id = m.home_team_api_id
    LEFT JOIN Team AS at
    ON at.team_api_id = m.away_team_api_id
    WHERE (
        m.home_team_api_id = 8633
        OR m.away_team_api_id = 8633
    )
"""

team_names_df = pd.read_sql_query(get_rma_matches, conn)